In [5]:
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile
import scipy.signal as sps
from scipy import signal
import glob
import librosa
import librosa.display
import cv2
from tqdm import tqdm
from keras.callbacks import TensorBoard
import os
import sys
import keras
import scipy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/AudioNoiseRemover
sys.path.append('/content/drive/MyDrive/AudioNoiseRemover')

In [9]:
"""!rm -r /content/drive/MyDrive/data/train/noisy
!rm -r /content/drive/MyDrive/data/train/original
os.mskdir('/content/drive/MyDrive/data/train/noisy')
os.mkdir('/content/drive/MyDrive/data/train/original') """

"!rm -r /content/drive/MyDrive/data/train/noisy\n!rm -r /content/drive/MyDrive/data/train/original\nos.mskdir('/content/drive/MyDrive/data/train/noisy')\nos.mkdir('/content/drive/MyDrive/data/train/original') "

In [10]:
import importlib
import modules
importlib.reload(modules)
#modules.WriteVoxFiles()
#modules.WriteUrbFiles()
#modules.ConvertAudio()

<module 'modules' from '/content/drive/MyDrive/AudioNoiseRemover/modules.py'>

In [ ]:
fftLength=255
hop_length=63
frame_length=16000

files_n = glob.glob('/content/drive/MyDrive/data/train/noisy/*wav')
files_o = glob.glob('/content/drive/MyDrive/data/train/original/*wav')
# print(len(files_o))
id = 0
for m in tqdm(range(0,500),desc="Progress.."):
  ystft_n = modules.ExtractSTFT(files_n[m],fftLength,hop_length,frame_length)
  ystft_x = modules.ExtractSTFT(files_o[m],fftLength,hop_length,frame_length)
  ystft_n = np.concatenate((ystft_n, ystft_n[:,0:8]), axis=1)
  ystft_x = np.concatenate((ystft_x, ystft_x[:,0:8]), axis=1)
  ystft_n = modules.reshape(ystft_n)
  ystft_x = modules.reshape(ystft_x)

  if(id==0):
    ystft_n_all = ystft_n.copy()
    ystft_x_all = ystft_x.copy()
    id = 1
  else:
    ystft_n_all = np.concatenate((ystft_n_all,ystft_n),axis=0)
    ystft_x_all = np.concatenate((ystft_x_all,ystft_x),axis=0)

  if(m<50):
    fig, ax = plt.subplots(1, 16, figsize=(16, 2))
    for n in range(0,16):
      if(n<8):
        ax[n].imshow(ystft_x[n,:,:,0])
      else:
        ax[n].imshow(ystft_n[n-8,:,:,0])
      ax[n].axis('off')
print(ystft_n_all.shape)

In [ ]:
model = modules.CNNmodel()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 16)         160       ['input_1[0][0]']             
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 128, 128, 16)         0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 16)         2320      ['leaky_re_lu[0][0]']         
                                                                                              

In [ ]:
# Dont Run
""" history = model.fit(ystft_n_all, ystft_x_all,
                epochs=120,
                batch_size=32,
                shuffle=True,
                callbacks=[TensorBoard(log_dir='tensorflow_logs', histogram_freq=0, write_graph=False)])
model.save('/content/drive/MyDrive/model.h5.h5')"""

" history = model.fit(ystft_n_all, ystft_x_all,\n                epochs=120,\n                batch_size=32,\n                shuffle=True,\n                callbacks=[TensorBoard(log_dir='tensorflow_logs', histogram_freq=0, write_graph=False)])\nmodel.save('/content/drive/MyDrive/model.h5.h5')"

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/model.h5')
y_pred = model.predict(ystft_n_all)

103/125 [=======================>......] - ETA: 0s

In [ ]:
!pip install numpy scipy matplotlib librosa tensorflow

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import librosa
from tensorflow.keras.models import load_model
import modules  # Make sure this custom module is accessible
import random

# Paths for the model and the folder containing audio files
model_path = '/content/drive/MyDrive/model.h5'
audio_folder = '/content/drive/MyDrive/data/train/noisy'  # Update this path

# Output directory
output_dir = '/content/drive/MyDrive/AudioNoiseRemover/output'
os.makedirs(output_dir, exist_ok=True)

# Load the model
model = load_model(model_path)

# Function to process audio files
def process_audio(file_path, sample_num):
    # Read audio file
    fs, x = wavfile.read(file_path)
    x = x.astype('float64')

    # Normalize input waveform
    x_normalized = x / np.array([x.max(), np.abs(x.min())]).max()

    # Process audio
    ystft_n = modules.ExtractSTFT(file_path, fftLength=255, hop_length=63, frame_length=16000)
    ystft_n = np.concatenate((ystft_n, ystft_n[:, 0:8]), axis=1)
    ystft_np = modules.reshape(ystft_n)
    y_pred = model.predict(ystft_np)
    ystft_np = modules.AssembleSTFT(y_pred)

    # Convert back to time domain
    noise_stft = librosa.stft(x, n_fft=255, hop_length=63)
    _, noisy_stft_phase = librosa.magphase(noise_stft)
    y = modules.InverseSTFT(ystft_np, noisy_stft_phase)
    y_normalized = y / np.array([y.max(), np.abs(y.min())]).max()
    y_final = (y_normalized * 32767).astype('int16')

    # Plot and save input waveform
    plt.figure(figsize=(12, 6))
    plt.subplot(2, 2, 1)
    t = np.arange(0, len(x) / fs, 1 / fs)
    plt.plot(t, x_normalized)
    plt.title('Input Waveform')
    plt.axis('off')

    # Plot and save output waveform
    plt.subplot(2, 2, 2)
    plt.plot(y_final)
    plt.title('Output Waveform')
    plt.axis('off')

    # Plot and save input spectrogram
    plt.subplot(2, 2, 3)
    ystft_n_abs = np.abs(ystft_n).T
    librosa.display.specshow(ystft_n_abs, sr=fs, hop_length=63)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Input Spectrogram')
    plt.axis('off')

    # Plot and save output spectrogram
    plt.subplot(2, 2, 4)
    ystft_np_abs = np.abs(ystft_np).T
    librosa.display.specshow(ystft_np_abs, sr=fs, hop_length=63)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Output Spectrogram')
    plt.axis('off')

    # Save the combined image
    combined_path = os.path.join(output_dir, f'sample_{sample_num}_combined.png')
    plt.savefig(combined_path, bbox_inches='tight')
    plt.close()

    print(f"Sample {sample_num} processed.")

# Get 20 random audio files from the folder
audio_files = os.listdir(audio_folder)
random_files = random.sample(audio_files, 50)

# Process each random audio file
for i, file_name in enumerate(random_files):
    file_path = os.path.join(audio_folder, file_name)
    process_audio(file_path, i + 1)


1/1 [==============================] - 0s 461ms/step
Sample 1 processed.
1/1 [==============================] - 0s 26ms/step
Sample 2 processed.
1/1 [==============================] - 0s 25ms/step
Sample 3 processed.
1/1 [==============================] - 0s 20ms/step
Sample 4 processed.
1/1 [==============================] - 0s 29ms/step
Sample 5 processed.
1/1 [==============================] - 0s 63ms/step
Sample 6 processed.
1/1 [==============================] - 0s 19ms/step
Sample 7 processed.
1/1 [==============================] - 0s 19ms/step
Sample 8 processed.
1/1 [==============================] - 0s 20ms/step
Sample 9 processed.
1/1 [==============================] - 0s 21ms/step
Sample 10 processed.
1/1 [==============================] - 0s 19ms/step
Sample 11 processed.
1/1 [==============================] - 0s 19ms/step
Sample 12 processed.
1/1 [==============================] - 0s 19ms/step
Sample 13 processed.
1/1 [==============================] - 0s 20ms/step
Sample 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import librosa
from tensorflow.keras.models import load_model
import modules  # Make sure this custom module is accessible

# Paths for the model and the file
model_path = '/content/drive/MyDrive/model.h5'
file_path = '/content/drive/MyDrive/AudioNoiseRemover/138.wav'  # Update this path

# Output directories
output_dir = '/content/drive/MyDrive/AudioNoiseRemover/output'
os.makedirs(output_dir, exist_ok=True)

# Load your model
model = load_model(model_path)

def process_audio(file_path):
    fname = os.path.basename(file_path)
    fs, x = wavfile.read(file_path)
    t = np.arange(0, len(x) / fs, 1 / fs)
    x_normalized = x / np.array([x.max(), np.abs(x.min())]).max()

    # Save input waveform
    plt.figure()
    plt.plot(t, x_normalized)
    plt.axis('off')
    input_waveform_path = os.path.join(output_dir, 'input_waveform.png')
    plt.savefig(input_waveform_path, bbox_inches='tight')
    plt.close()

    # Process audio
    ystft_n = modules.ExtractSTFT(file_path, fftLength=255, hop_length=63, frame_length=16000)
    ystft_n = np.concatenate((ystft_n, ystft_n[:, 0:8]), axis=1)
    ystft_np = modules.reshape(ystft_n)

    y_pred = model.predict(ystft_np)
    ystft_np = modules.AssembleSTFT(y_pred)

    # Convert back to time domain
    x = x.astype('float64')
    noise_stft = librosa.stft(x, n_fft=255, hop_length=63)
    _, noisy_stft_phase = librosa.magphase(noise_stft)
    y = modules.InverseSTFT(ystft_np, noisy_stft_phase)
    y_normalized = y / np.array([y.max(), np.abs(y.min())]).max()
    y_final = (y_normalized * 32767).astype('int16')

    # Save output waveform
    plt.figure()
    plt.plot(y_final)
    plt.axis('off')
    output_waveform_path = os.path.join(output_dir, 'output_waveform.png')
    plt.savefig(output_waveform_path, bbox_inches='tight')
    plt.close()

    # Save the denoised audio file
    output_audio_path = os.path.join(output_dir, f'processed_{fname}')
    wavfile.write(output_audio_path, fs, y_final)

    print(f"Process complete. Output saved to {output_dir}")

# Call the processing function
process_audio(file_path)


1/1 [==============================] - 1s 1s/step
Process complete. Output saved to /content/drive/MyDrive/AudioNoiseRemover/output
